In [1]:
%reload_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

import numpy as np
import healpy as hp
from healpy.newvisufunc import projview, newprojplot
from astropy.io import fits
from pprint import pprint
from tqdm import tqdm
import pickle
import corner
import os

from scipy import optimize
from scipy.stats import poisson

import jax
import jax.numpy as jnp

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc_file('../utils/matplotlibrc')

from utils import ed_fcts_amarel as ef
from utils import create_mask as cm
from utils import ed_plotting as eplt

# load GPU
gpu_id = '2'
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'
os.environ['CUDA_VISIBLE_DEVICES'] = gpu_id

from models.poissonian_gp import EbinPoissonModel

/home/edr76/.conda/envs/jax/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load SVI fit results (these should be the only parameters that you are loading)
sim_seeds = np.arange(1000,1010)

# name of the synthetic directory
sim_name = 'canon_g1p2_ola_v2'

# load SVI fit results (these should be the only parameters that you are loading)
sim_id = 7.1234567
temp_id = 5.23457
gp_id = 1.16
blg_id = -1
mod_id = 11
svi_id = 7840 # (24,25) =  (no outer roi, outer roi)
sim_seed = 1000
svi_seed = 0

In [3]:
# load GPU
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'
os.environ['CUDA_VISIBLE_DEVICES'] = gpu_id

data_dir = ef.load_data_dir(sim_name)
os.system("mkdir -p "+data_dir)

# Load the simulated templates
ebin = 10
temp_dict = np.load(data_dir + 'all_templates_ebin' + str(ebin)  + '.npy', allow_pickle=True).item()

In [4]:
fit_filename, module_name = ef.generate_fit_filename_from_ids(sim_id, temp_id, gp_id, blg_id, mod_id, svi_id, sim_seed, svi_seed)
fit_dir = data_dir + 'fits/' + fit_filename + '/'
ef.list_files(fit_dir)

sys.path.append(fit_dir)
import importlib 
module = importlib.import_module(module_name)

# Load all the variables from the module
globals().update(vars(module))

/
    settings_7p1234567_5p23457_1p16_-1_11_7840_1000_0.py
    ebin10_svi_res_0.1_20000_mvn_8_1000_0.p
    ebin_old10_smp_svi_0.1_20000_mvn_8_1000_0.p
    __init__.py
    summary.txt
    ebin10_smp_svi_0.1_20000_mvn_8_1000_0.p
__pycache__/
    settings_7p1234567_5p23457_1p16_-1_11_7840_1000_0.cpython-311.pyc


In [5]:
# load model using stored parameters
ebinmodel = EbinPoissonModel(
        # important parameters
        rig_temp_list = rig_temp_list,
        hyb_temp_list = hyb_temp_list,
        var_temp_list = var_temp_list,
        is_gp = is_gp,
        gp_deriv = gp_deriv,
        data_file = data_file,
        rig_temp_sim = rig_temp_sim,
        hyb_temp_sim = hyb_temp_sim,
        var_temp_sim = var_temp_sim,
        is_custom_blg = is_custom_blg,
        custom_blg_id = custom_blg_id,
        sim_seed = sim_seed,
        Nu = Nu,
        u_option = u_option,
        u_grid_type = u_grid_type,
        u_weights = u_weights,
        Np = Np,
        p_option = p_option,
        Nsub = Nsub,

        # default parameters
        ebin = ebin,
        is_float64 = is_float64,
        debug_nans = debug_nans,
        no_ps_mask = no_ps_mask,
        p_grid_type = p_grid_type,
        p_weights = p_weights,
        gp_kernel = gp_kernel,
        gp_params = gp_params,
        gp_scale_option = gp_scale_option,
        monotonicity_hyperparameter = monotonicity_hyperparameter,
        nfw_gamma = nfw_gamma,
        blg_names = blg_names,
        dif_names = dif_names,
        )
# configure model, run SVI, and generate samp 
ebinmodel.config_model(ebin=ebin)
mask_p = ebinmodel.mask_roi_arr[ebin]
mask = cm.make_mask_total(
        nside=128,
        mask_ring=True,
        outer = 20.,
        inner = 0.,
    )

In [6]:
def truncate_samples(samples, n_samples):
    for key in samples.keys():
        samples[key] = samples[key][:n_samples]
    return samples

In [7]:
def build_mod_id(dif_models, mod_data):
    models = ['o', 'a', 'f']
    num_dict = {m : str(i+1) for i, m in enumerate(models)}

    mod_id = [num_dict[i] for i in dif_models]
    mod_id.append(num_dict[mod_data])
    mod_id = ''.join(mod_id)
    return int(mod_id)

mod_ids = []

dif_mods = [['o'], ['a'], ['f']]
data_mods = ['o', 'a', 'f']
gpu_id = '0'

txt = lambda x: ('\"' + str(x) + '\"')

for df in tqdm(dif_mods, desc = 'dif_mod', position = 0):
    for dm in tqdm(data_mods, desc = 'dat_mod', position = 1, leave = False):
        mod_ids.append(build_mod_id(df, dm))

# print('-----------------------------------')
# dif_mods = [['a', 'f'], ['o', 'f'], ['o', 'a']]
# data_mods = ['o', 'a', 'f']

# for i in tqdm(range(len(dif_mods)), desc = 'idx '):
#     df = dif_mods[i]
#     dm = data_mods[i]
#     mod_ids.append(build_mod_id(df, dm))

# print('-----------------------------------')
# dif_mods = [['o', 'a', 'f']]
# data_mods = ['o', 'a', 'f']

# for dm in tqdm(data_mods):
#     df = dif_mods[0]
#     mod_ids.append(build_mod_id(df, dm))

# mod_ids.reverse()
print(mod_ids)

dif_mod: 100%|██████████| 3/3 [00:00<00:00, 180.11it/s]

[11, 12, 13, 21, 22, 23, 31, 32, 33]


In [8]:
# cartesian map of masks to keep track of masking for plots
n_pixels = 160 # 160 is our best choice
mask_p = ebinmodel.mask_roi_arr[ebin]
mask = cm.make_mask_total(
        nside=128,
        mask_ring=True,
        outer = 20.,
        inner = 0.,
    )
mask_map = np.zeros((~ebinmodel.mask_roi_arr[ebin]).sum())
mask_map_cart = ef.healpix_to_cart(mask_map, ebinmodel.mask_roi_arr[ebin], n_pixels = n_pixels, nside = 128, nan_fill = True) # doesn't matter what mask used


In [9]:
# We'll start with the difficult stuff
# We'll start by making the single model fits

def build_mod_id(dif_models, mod_data):
    models = ['o', 'a', 'f']
    num_dict = {m : str(i+1) for i, m in enumerate(models)}

    mod_id = [num_dict[i] for i in dif_models]
    mod_id.append(num_dict[mod_data])
    mod_id = ''.join(mod_id)
    return int(mod_id)

temp_dict_list = {}
temp_sample_dict_list = {}
temp_sample_dict_cmask_list = {}
exp_gp_samples_cart_list = {}
gp_true_cart_list = {}
tot_samples_cart_list = {}
model_residuals_cart_list = {}

for mod_id in tqdm(mod_ids, desc = 'mod_id', position = 0):
    data_models = ['o', 'a', 'f']
    mod_data = data_models[int(str(mod_id)[-1]) - 1]
    sim_name = data_file = 'canon_g1p2_{}la_v2'.format(mod_data)
    data_dir = ef.load_data_dir(sim_name)

    temp_dict = np.load(data_dir + 'all_templates_ebin' + str(ebin)  + '.npy', allow_pickle=True).item()
    temp_dict_list[mod_id] = temp_dict

    aug_temp_sample_dict = []
    aug_temp_sample_dict_cmask = []
    aug_exp_gp_samples_cart = []
    aug_gp_true_cart = []
    aug_tot_samples_cart = []
    aug_model_residuals_cart = []

    for sim_seed in tqdm(sim_seeds, desc = 'sim_seed', position = 1, leave = False):
        str_sim_seed = str(sim_seed)
        
        # load file names and data
        fit_filename, module_name = ef.generate_fit_filename_from_ids(sim_id, temp_id, gp_id, blg_id, mod_id, svi_id, sim_seed, svi_seed)
        fit_dir = data_dir + 'fits/' + fit_filename + '/'
        # ef.list_files(fit_dir)

        # sys.path.append(fit_dir)
        # import importlib 
        # module = importlib.import_module(module_name)

        # # Load all the variables from the module
        # globals().update(vars(module))

        file_name = ('ebin' + str_ebin + '_smp_svi_' + 
                    str_lr + '_' + str_n_steps + '_' + 
                        str_guide + '_' + str_num_particles + '_' + 
                        str_sim_seed + '_' + str_svi_seed + '.p')
        samples_dict, _ = pickle.load(open(fit_dir + file_name, 'rb'))

        # generate temp_sample_dict (less samples)
        all_temp_names = ['iso', 'psc', 'bub', 'pib', 'ics', 'blg', 'nfw', 'dsk', 'gp']
        names = list(samples_dict.keys())
        temp_sample_dict = {k: samples_dict[k] for k in all_temp_names if k in names}
        temp_sample_dict_cmask = {k: samples_dict[k + '_cmask'] for k in all_temp_names if k in names}

        temp_sample_dict = truncate_samples(temp_sample_dict, 100)
        temp_sample_dict_cmask = truncate_samples(temp_sample_dict_cmask, 100)

        # save temp_sample_dict
        pickle.dump(
            (temp_sample_dict, temp_sample_dict_cmask), 
            open('plotting_data/single/temp_sample_dict_{}_{}.p'.format(str(mod_id), str(sim_seed)), 'wb'))

        sim_file_name = ef.make_pseudodata_file(ebinmodel.temp_names_sim, ebinmodel.sim_data_dir, create_dir = False, return_name=True, sim_seed=sim_seed, 
                                        is_custom_blg=is_custom_blg, custom_blg_id=custom_blg_id)
        ebinmodel.counts = jnp.array(np.load(sim_file_name), dtype = jnp.float32)

        # cartesian gp samples
        exp_gp_samples = temp_sample_dict_cmask['gp']
        exp_gp_samples_cart = ef.multi_healpix_to_cart(exp_gp_samples, mask, n_pixels = n_pixels, nside = 128, progress_bar = False)

        # create gp_true from scratch
        temp_names_sim = rig_temp_sim + hyb_temp_sim + var_temp_sim # imported from settings file
        ebinmodel.load_templates(temp_names_sim, blg_names, dif_names)
        gp_true = ( temp_dict['S_blg'] * ebinmodel.blg_temps[0].at_bin(ebin, mask) + temp_dict['S_nfw'] * ebinmodel.nfw_temp.get_NFW2_template(gamma = temp_dict['gamma'])[~mask] )

        # 1D slice of total rate
        tot_samples = jnp.zeros(np.sum(~mask))
        tot_names = list(temp_sample_dict_cmask.keys())
        for tot_name in tot_names:
            tot_samples += temp_sample_dict_cmask[tot_name]
        tot_samples_cart = ef.multi_healpix_to_cart(tot_samples, mask, n_pixels = n_pixels, nside = 128, progress_bar = False)

        sim_samples = jnp.zeros(np.sum(~mask))
        for sim_name in temp_names_sim:
            sim_samples += temp_dict[sim_name][~mask]

        # 1D slice of residual posterior rate samples relative to truth
        model_residuals = tot_samples - sim_samples
        model_residuals_cart = ef.multi_healpix_to_cart(model_residuals, mask, n_pixels=n_pixels, nside = 128, progress_bar = False)

        # 1D slice of residual data relative to posterior samples
        rng_key = jax.random.PRNGKey(mod_id)
        rng_key, key = jax.random.split(rng_key)
        poisson_samples = jax.random.poisson(key, tot_samples)
        data_residuals = ebinmodel.counts[ebin][~mask] - poisson_samples
        data_residuals_cart = ef.multi_healpix_to_cart(data_residuals, mask, n_pixels = n_pixels, nside = 128, progress_bar = False)

        # combine samples corresponding to different datasets
        if len(aug_temp_sample_dict) == 0:
            aug_temp_sample_dict = temp_sample_dict
            aug_temp_sample_dict_cmask = temp_sample_dict_cmask
            aug_exp_gp_samples_cart = exp_gp_samples_cart
            aug_gp_true_cart = gp_true
            aug_tot_samples_cart = tot_samples_cart
            aug_model_residuals_cart = model_residuals_cart
        else:
            for key in temp_sample_dict.keys():
                aug_temp_sample_dict[key] = np.concatenate((aug_temp_sample_dict[key], temp_sample_dict[key]))
                aug_temp_sample_dict_cmask[key] = np.concatenate((aug_temp_sample_dict_cmask[key], temp_sample_dict_cmask[key]))
            aug_exp_gp_samples_cart = np.concatenate((aug_exp_gp_samples_cart, exp_gp_samples_cart))
            aug_gp_true_cart = np.concatenate((aug_gp_true_cart, gp_true))
            aug_tot_samples_cart = np.concatenate((aug_tot_samples_cart, tot_samples_cart))
            aug_model_residuals_cart = np.concatenate((aug_model_residuals_cart, model_residuals_cart))

    pickle.dump(
        (aug_temp_sample_dict, aug_temp_sample_dict_cmask, aug_exp_gp_samples_cart, aug_gp_true_cart, aug_tot_samples_cart, aug_model_residuals_cart), 
        open('plotting_data/multi/aug_temp_sample_dict_{}.p'.format(str(mod_id)), 'wb'))
        
    temp_sample_dict_list[mod_id] = aug_temp_sample_dict
    temp_sample_dict_cmask_list[mod_id] = aug_temp_sample_dict_cmask
    exp_gp_samples_cart_list[mod_id] = aug_exp_gp_samples_cart
    gp_true_cart_list[mod_id] = aug_gp_true_cart
    tot_samples_cart_list[mod_id] = aug_tot_samples_cart
    model_residuals_cart_list[mod_id] = aug_model_residuals_cart

pickle.dump(
    (temp_sample_dict_list, temp_sample_dict_cmask_list, exp_gp_samples_cart_list, gp_true_cart_list, tot_samples_cart_list, model_residuals_cart_list, mask, mask_p, mask_map_cart), 
    open('plotting_data/multi/all_mismodelling_data.p', 'wb'))

mod_id: 100%|██████████| 9/9 [1:01:06<00:00, 407.41s/it]


In [10]:
temp_dict_list = {}
sim_cart_list = {}

for mod_id in tqdm(mod_ids, desc = 'mod_id', position = 0):
    data_models = ['o', 'a', 'f']
    mod_data = data_models[int(str(mod_id)[-1]) - 1]
    sim_name = 'canon_g1p2_{}la_v2'.format(mod_data)
    data_dir = ef.load_data_dir(sim_name)

    temp_dict = np.load(data_dir + 'all_templates_ebin' + str(ebin)  + '.npy', allow_pickle=True).item()
    temp_dict_list[mod_id] = temp_dict

    sim_samples = jnp.zeros(np.sum(~mask))
    for sim_name in temp_names_sim:
        sim_samples += temp_dict[sim_name][~mask]

    sim_cart = ef.healpix_to_cart(sim_samples, mask, n_pixels = n_pixels, nside = 128)
    sim_cart_list[mod_id] = sim_cart

pickle.dump(
    (temp_dict_list, sim_cart_list), 
    open('plotting_data/multi/temp_dict_list.p', 'wb'))

mod_id:   0%|          | 0/9 [00:00<?, ?it/s]

mod_id: 100%|██████████| 9/9 [00:00<00:00, 17.40it/s]


In [11]:
# counts_list = {}
# raw_cart_list = {}

# for mod_id in tqdm(mod_ids, desc = 'mod_id', position = 0):
#     data_models = ['o', 'a', 'f']
#     mod_data = data_models[int(str(mod_id)[-1]) - 1]
#     sim_name = 'canon_g1p2_{}la'.format(mod_data)
#     data_dir = ef.load_data_dir(sim_name)

#     sim_file_name = ef.make_pseudodata_file(ebinmodel.temp_names_sim, ebinmodel.sim_data_dir, create_dir = False, return_name=True, sim_seed=sim_seed, 
#                                     is_custom_blg=is_custom_blg, custom_blg_id=custom_blg_id)
#     counts = jnp.array(np.load(sim_file_name), dtype = jnp.float32)
#     raw_cart = ef.healpix_to_cart(ebinmodel.counts[ebin][~mask], mask, n_pixels = n_pixels, nside = 128)
    
#     counts_list[mod_id] = counts
#     raw_cart_list[mod_id] = raw_cart

# pickle.dump(
#     (counts_list, raw_cart_list), 
#     open('plotting_data/multi/counts_list.p', 'wb'))


In [12]:
gp_true.shape

(5938,)

In [13]:
temp_dict_list[11].keys()

dict_keys(['iso', 'psc', 'bub', 'pib', 'ics', 'blg', 'nfw', 'S_iso', 'S_psc', 'S_bub', 'S_pib', 'S_ics', 'S_blg', 'S_nfw', 'gamma'])

In [14]:
def build_mod_id(dif_models, mod_data):
    models = ['o', 'a', 'f']
    num_dict = {m : str(i+1) for i, m in enumerate(models)}

    mod_id = [num_dict[i] for i in dif_models]
    mod_id.append(num_dict[mod_data])
    mod_id = ''.join(mod_id)
    return int(mod_id)